# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
# импорт библиотеки Pandas и открытие файла 'data.csv'
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# просмотр общей информации о файле
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# подсчет пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# подсчет дубликатов
data.duplicated().sum()


54

**Вывод**

В ходе изучения общей информации о файле были выявлены следующие проблемы в данных:
- Отрицательные значения трудового стажа в столбце 'days_employed';
- Пропущенные значения в столбцах 'days_employed' и 'total_income';
- Дубликаты.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# приведение данных в столбце 'days_employed' к значениям по модулю
data['days_employed'] = data['days_employed'].abs()

# замена пропусков в столбце 'days_employed'
data.loc[data['days_employed'].isna(),'days_employed']  = data['days_employed'].median()

# проверка столбца
data['days_employed'].isna().sum()

#замена пропусков в столбце 'total_income'
data.loc[data['total_income'].isna(),'total_income']  = data['total_income'].median()

# проверка столбца
data['total_income'].isna().sum()


0

**Вывод**

Пропущенные значения в столбцах 'days_employed' и 'total_income'т.е. общий трудовой стаж и ежемесячный доход, вероятно, возникли по причине отсутствия у клиентов трудового стажа и соответственно дохода, либо эти данные просто не были указаны при заполнении анкет, также не исключена техническая ошибка.
Данные пропуски были заполнены медианными значениями трудового стажа для столбца 'days_employed' и ежемесячного дохода для столбца 'total_income' с помощью метода isna().

### Замена типа данных

In [6]:
# замена типа данных с float на int методом astype
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')


**Вывод**

Согласно документации к данным: 
- 'days_employed'  - общий трудовой стаж в днях;
- 'total_income' - ежемесячный доход.

Значения в этих столбцах должны быть целыми, поэтому тип данных был преобразован из вещественного в целочисленный.

### Обработка дубликатов

In [7]:
# удаление дубликатов методом drop_duplicates(), аргумент drop со значением True,
# чтобы не создавать столбец со старыми значениями индексов
data = data.drop_duplicates().reset_index(drop=True)

# проверка таблицы
data.duplicated().sum()

0

**Вывод**

Явные дубликаты, которые могли возникнуть по причине влияния человеческого фактора, например, при копировании данных, удалены и не будут влиять на результаты анализа.

### Лемматизация

In [8]:
# импорт pymystem3
from pymystem3 import Mystem
m = Mystem() 

# получение уникальных значений столбца 'purpose'
purpose_unique = data['purpose'].unique()

# объдеинение в строку
purpose_str = ' '.join(purpose_unique)

# лемматизация
purpose_lem = m.lemmatize(purpose_str)

# подсчет значений в списке
from collections import Counter

# вывод на экран количества каждой леммы
print(Counter(purpose_lem))

# проходимся по каждой строке, лемматизируем и на основе полученных лемм создаем столбец с новыми категориями
def purpose_funс(cell):
    categories = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'строительство', 'ремонт']
    cell_lem = m.lemmatize(cell['purpose'])
    
    for category in categories:
        if category in cell_lem:
            return category
    return 'прочее'    

# создание столбца с новыми категориями
data['purpose_total'] = data.apply(purpose_funс, axis=1)

# вывод на экран столбцов 'purpose','purpose_total'
data[['purpose','purpose_total']].head(10)

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


,purpose,purpose_total
0,покупка жилья,прочее
1,приобретение автомобиля,автомобиль
2,покупка жилья,прочее
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,прочее
6,операции с жильем,прочее
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,прочее


**Вывод**

В ходе проведенных преобразований - поиск уникальных значений столбца 'purpose', лемматизация, определение количества каждой из лемм, стало возможным выделить 7 основных целей получения кредита.

### Категоризация данных

Для целей анализа, чтобы проследить зависимость возврата кредита в срок, нам необходимо выделить в файле категории в зависимости от:
- количества детей;
- семейного положения;
- уровня дохода.
 
 Для того, чтобы оптимально хранить информацию о категориях, создадим отдельные файлы-словари.

#### Количество детей

In [9]:
# создание словаря с данными по количеству детей
children = data['children']
children = children.drop_duplicates()
children


0        1
2        0
3        3
8        2
291     -1
294      4
606     20
3978     5
Name: children, dtype: int64

Обработаем артефакты, которые встретились в столбце 'children', а именно:
- отрицательное количество детей,для этого применим метод 'abs';
- количество детей равное 20 заменим на 2, т.к. на num pad цифры о и 2 рядом, скорее всего произошла опечатка. 

In [10]:
# избавимся от отрицательных значений и заменим количество 20 на 2
data['children'] = data['children'].abs().replace(20, 2)

# удаление дубликатов 
children = data['children'].drop_duplicates()
children

0       1
2       0
3       3
8       2
294     4
3978    5
Name: children, dtype: int64

Теперь создаем категории по количеству детей:
- нет детей;
- 1-2 ребенка;
- много детей.

In [11]:
# создаем функцию, которая будет проходиться по всем строкам исходной таблицы и проверять значения из столбца 'children' 
# на соответствие выделенным категориям
def children_func(children):
    if children==0:
        return 'нет детей'
    elif 1 >= children <= 2:
        return '1-2 ребенка'
    return 'много детей'

# создаем столбец с категорями по количеству детей
data['children_group'] = data['children'].apply(children_func)
data[['children','children_group']].head()

,children,children_group
0,1,1-2 ребенка
1,1,1-2 ребенка
2,0,нет детей
3,3,много детей
4,0,нет детей


#### Семейное положение

In [12]:
# создание словаря с данными категорий семейного положения
family = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True) 
family

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


#### Уровень дохода

In [13]:
# создание списка критериев уровня дохода
labels=['низкий', 'средний', 'высокий']

# разбивка дохода на группы и создание нового столбца с этими группами с помощью метода qcut
data['income_group'] = pd.qcut(data['total_income'], 3, labels=labels)


**Вывод**

Мы разбили основные показатели, влияющие на возврат кредита в срок, на категории и далее можно группировать данные по этим категорям, чтобы ответить на основные вопросы исследования.

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
# группировка данных по категориям количества детей и 
# подсчет коэффициента невозврата кредита(как среднего значения по столбцу 'debt') и кол-ва невозвратов
report_1 = data.groupby('children_group').agg({'debt':['mean', 'count']})
report_1

debt       
                    mean  count
children_group                 
1-2 ребенка     0.091639   4856
много детей     0.092903   2508
нет детей       0.075353  14107

**Вывод**

На основании созданного отчета можно сделать вывод, что самая большая доля невозвратов среди категории людей, у которых много детей (больше 2-х), наименьшая доля невозвратов среди бездетных клиентов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
# группировка данных по категориям семейного положения
report_2 = data.groupby('family_status').agg({'debt':['mean', 'count']})
report_2

debt       
                           mean  count
family_status                         
Не женат / не замужем  0.097509   2810
в разводе              0.071130   1195
вдовец / вдова         0.065693    959
гражданский брак       0.093202   4163
женат / замужем        0.075421  12344

**Вывод**


Самая большая доля невозвратов среди не женатых/не замужних людей, а самые ответственные заемщики - вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
# группировка данных по категориям уровня дохода
report_3 = data.groupby('income_group').agg({'debt':['mean', 'count']})
report_3

debt      
                  mean count
income_group                
низкий        0.081179  7157
средний       0.086349  7157
высокий       0.075730  7157

**Вывод**

Люди с высоким уровнем дохода чаще возвращают кредиты, доля невозврата по ним самая маленькая, а вот люди со средним доходом невозвращают кредиты даже чаще, чем люди с низким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
# группировка данных по целям кредита
report_4 = data.groupby('purpose_total').agg({'debt':['mean', 'count']})
report_4


debt      
                   mean count
purpose_total                
автомобиль     0.093547  4308
недвижимость   0.074610  6353
образование    0.092177  4014
прочее         0.070835  3854
ремонт         0.057661   607
свадьба        0.079657  2335

**Вывод**

Самая большая доля невозвратов среди людей, взявших кредит на приобретение автомобиля и образование, а вот люди, взвшие кредит на ремонт, возвращают деньги чаще.

## Шаг 4. Общий вывод


    
В исходных данных были выявлены проблемы, перед исследованием была осуществлена предобработка:
- пропущенные значения дней трудового стажа и ежемесячного дохода были заменены на медианные значения;
- был преобразован тип данных из вещественного в целочисленный;
- были удалены дубликаты.

Данные о заемщиках были объединены в категории и было выявлено, что на факт погашения кредита в срок влияет:
- количество детей;
- семейное положение;
- уровень дохода;
- цель кредита.
    
Таким образом в ходе исследования было установлено:
    
1. На многодетных заемщиков (больше 2-х детей) приходится самая большая доля невозвратов, а заемщики, у которых нет детей ведут себя более ответственно, доля невозвратов по ним наименьшая. 
 
    
2. В зависимотси от семейного положения самая большая доля невозвратов среди не женатых/не замужних людей, а самые ответственные заемщики - вдовцы.    
     
    
3. В зависимости от уровня дохода было установлено, что люди с высоким уровнем дохода чаще возвращают кредиты, доля невозвратов по ним самая маленькая, а вот люди со средним доходом невозвращают кредиты даже чаще, чем люди с низким доходом.
    
    
4. Зависимость от целей кредита (всего их было выделено 7: автомобиль, недвижимость, образование, ремонт, свадьба и прочее) следующая: самая большая доля невозвратов среди людей, взявших кредит на приобретение автомобиля и образование, а вот люди, взвшие кредит на ремонт, возвращают деньги чаще.
    